In [ ]:
# Colab+so-vits

# 配置Conda
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py39_4.11.0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_4.11.0-Linux-x86_64.sh
!bash Miniconda3-py39_4.11.0-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.insert(0, "/usr/local/lib/python3.9/site-packages/") # 加入环境变量

!conda create -n vits python=3.9.1 -y # 创建独立环境
!activate vits
!python -V

# 下载项目并安装依赖
!git clone https://github.com/Plutoisy/so-vits-svc
%cd /content/so-vits-svc
!pip install -r requirements.txt

!pip uninstall numpy -y
!pip install numpy==1.23.5

# 下载预模型
!wget https://huggingface.co/Plutoisy/pt_file/resolve/main/checkpoint_best_legacy_500.pt -P hubert/

In [ ]:
# 创建目录
!mkdir raw
!mkdir dataset_raw

# 挂载Google Drive
from google.colab import drive
drive.mount("/content/drive")

# 从Google Drive拷贝压缩包
!cp /content/drive/MyDrive/test.zip /content/so-vits-svc/dataset_raw/test.zip
!unzip /content/so-vits-svc/dataset_raw/test.zip -d /content/so-vits-svc/dataset_raw/

# 重采样至44100hz
!python resample.py

# 划分数据集+生成配置文件
!python preprocess_flist_config.py

# 生成hubert和f0
!python preprocess_hubert_f0.py

# 备份已处理数据
!zip -r dataset.zip /content/so-vits-svc/dataset
!cp /content/so-vits-svc/dataset.zip /content/drive/MyDrive/test/
!cp configs/config.json /content/drive/MyDrive/test/
!cp filelists/train.txt /content/drive/MyDrive/test/
!cp filelists/val.txt /content/drive/MyDrive/test/

In [ ]:
# 首次训练使用预模型
pre_pth = False #@param {type:"boolean"}
if pre_pth:
    !wget https://huggingface.co/Plutoisy/pt_file/resolve/main/D_0.pth -P logs/44k/
    !wget https://huggingface.co/Plutoisy/pt_file/resolve/main/G_0.pth -P logs/44k/

# 继续训练使用已有模型
Save_to_drive = False #@param {type:"boolean"}
if Save_to_drive:
    !rm -rf /content/so-vits-svc/logs/44k
    !cp /content/drive/MyDrive/D_2400.pth /content/so-vits-svc/logs/44k/D_2400.pth
    !cp /content/drive/MyDrive/G_2400.pth /content/so-vits-svc/logs/44k/G_2400.pth

# 训练
!pip uninstall numpy -y
!pip install numpy==1.22.0 # 降级numpy训练

!python train.py -c configs/config.json -m 44k

# 备份模型
!cp /content/so-vits-svc/logs/44k/D_2400.pth /content/drive/MyDrive/test/
!cp /content/so-vits-svc/logs/44k/G_2400.pth /content/drive/MyDrive/test/

# 测试（超过10秒无法转换）
!cp /content/drive/MyDrive/test/10000.ogg /content/so-vits-svc/raw/
!python inference_main.py -m "logs/44k/G_2400.pth" -c "configs/config.json" -n "10000.ogg" -s test # -s为数据集名称

# 查看并复制结果
!ls -l -f /content/so-vits-svc/results
!cp /content/so-vits-svc/results/10000.ogg_0key_test.flac /content/drive/MyDrive/test/